In [ ]:
import tensorflow as tf
import numpy as np
import os

In [ ]:
def load_data(data_dir,kind):
    """
    kind -  ANN / LSTM
    """
    data = np.load(os.path.join(data_dir, "%s.npz"%kind))
    train_x, train_y = data["train_x"], data["train_y"]
    val_x, val_y = data["val_x"], data["val_y"]
    return train_x, train_y, val_x, val_y

In [ ]:
def batch_create(i, x_train, y_train, batch_size):
    x_batch = x_train[(i*batch_size) : ((i+1)*(batch_size))]
    y_batch = y_train[(i*batch_size) : ((i+1)*(batch_size))]
    return x_batch, y_batch

In [ ]:
train_x, train_y, val_x, val_y = load_data("/Data/Task_1", "LSTM")

In [ ]:
print train_x.shape

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 50, 2048])
y_ = tf.placeholder(tf.float32, shape=[None, 5])

batch_size = 49

In [ ]:
rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(128)
outputs, state = tf.nn.dynamic_rnn(rnn_cell, x, dtype=tf.float32)
del state

In [ ]:
outputs.shape

In [ ]:
flatten_out = tf.reshape(outputs,[-1, 50*128])

In [ ]:
W1 = tf.get_variable("W1", shape=[50*128,128], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.zeros([128]))

In [ ]:
W2 = tf.get_variable("W2", shape=[128,5], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.zeros([5]))

In [ ]:
z1 = tf.matmul(flatten_out,W1) + b1
a1 = tf.nn.relu(z1, name=None)
y = tf.matmul(a1,W2) + b2

In [ ]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [ ]:
train_step = tf.train.AdamOptimizer(learning_rate = 1e-4).minimize(cross_entropy)

In [ ]:
batch_size = 49
num_batch = np.size(train_x,0) / batch_size
print num_batch
alpha = tf.Variable(0.01)

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epochs in range(20):
        for iteration in range(num_batch):
            x_batch , y_batch = batch_create(iteration, train_x, train_y, batch_size)
            train_step.run(feed_dict={x: x_batch, y_: y_batch})        
        print("train_accuracy = %f " %(accuracy.eval(feed_dict={x: train_x, y_: train_y})))
        print("val_accuracy = %f" %(accuracy.eval(feed_dict={x: val_x, y_: val_y})))